In [9]:

#taken from the several iciar 2018 implementations

import argparse
import numpy as np
import cv2
from scipy import ndimage
import os
from os.path import basename, join, exists
from os import makedirs
#from threaded_generator import threaded_generator
from time import time
import sys
np.random.seed(13)

PATCH_SIZES = [224, 224]
SCALES = [0.5]

DEFAULT_INPUT_DIR = "data/train"
DEFAULT_PREPROCESSED_ROOT = "data/preprocessed/train"

PATCHES_PER_IMAGE = 20
AUGMENTATIONS_PER_IMAGE = 50
COLOR_LO = 0.7
COLOR_HI = 1.3
BATCH_SIZE = 16     # decrease if necessary

NUM_CACHED = 160


In [2]:
def normalize_staining(img):
   
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    h, w, c = img.shape
    img = img.reshape(h * w, c)
    OD = -np.log((img.astype("uint16") + 1) / Io)
    ODhat = OD[(OD >= beta).all(axis=1)]
    W, V = np.linalg.eig(np.cov(ODhat, rowvar=False))

    Vec = -V.T[:2][::-1].T  
    That = np.dot(ODhat, Vec)
    phi = np.arctan2(That[:, 1], That[:, 0])
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = HE.T
    Y = OD.reshape(h * w, c).T

    C = np.linalg.lstsq(HE, Y)
    maxC = np.percentile(C[0], 99, axis=1)

    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(-np.dot(HERef, C))
    Inorm = Inorm.T.reshape(h, w, c).clip(0, 255).astype("uint8")

    return Inorm


def hematoxylin_eosin_aug(img, low=0.7, high=1.3, seed=None):
    D = np.array([[1.88, -0.07, -0.60],
                  [-1.02, 1.13, -0.48],
                  [-0.55, -0.13, 1.57]])
    M = np.array([[0.65, 0.70, 0.29],
                  [0.07, 0.99, 0.11],
                  [0.27, 0.57, 0.78]])
    Io = 240

    h, w, c = img.shape
    OD = -np.log10((img.astype("uint16") + 1) / Io)
    C = np.dot(D, OD.reshape(h * w, c).T).T
    r = np.ones(3)
    r[:2] = np.random.RandomState(seed).uniform(low=low, high=high, size=2)
    img_aug = np.dot(C, M) * r

    img_aug = Io * np.exp(-img_aug * np.log(10)) - 1
    img_aug = img_aug.reshape(h, w, c).clip(0, 255).astype("uint8")
    return img_aug

def zoom_aug(img, zoom_var, seed=None):
    scale = np.random.RandomState(seed).uniform(low=1 / zoom_var, high=zoom_var)
    resized_img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    return resized_img

def get_crops(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    crops = []
    for _ in range(n):
        top = np.random.randint(low=0, high=h - size + 1)
        left = np.random.randint(low=0, high=w - size + 1)
        crop = img[top: top + size, left: left + size].copy()
        crop = np.rot90(crop, np.random.randint(low=0, high=4))
        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def get_crops_free(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    d = int(np.ceil(size / np.sqrt(2)))
    crops = []
    for _ in range(n):
        center_y = np.random.randint(low=0, high=h - size + 1) + size // 2
        center_x = np.random.randint(low=0, high=w - size + 1) + size // 2
        m = min(center_y, center_x, h - center_y, w - center_x)
        if m < d:
            max_angle = np.pi / 4 - np.arccos(m / d)
            top = center_y - m
            left = center_x - m
            precrop = img[top: top + 2 * m, left: left + 2 * m]
        else:
            max_angle = np.pi / 4
            top = center_y - d
            left = center_x - d
            precrop = img[top: top + 2 * d, left: left + 2 * d]

        precrop = np.rot90(precrop, np.random.randint(low=0, high=4))
        angle = np.random.uniform(low=-max_angle, high=max_angle)
        precrop = ndimage.rotate(precrop, angle * 180 / np.pi, reshape=False)

        precrop_h, precrop_w, _ = precrop.shape
        top = (precrop_h - size) // 2
        left = (precrop_w - size) // 2
        crop = precrop[top: top + size, left: left + size]

        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def norm_pool(features, p=3):
    return np.power(np.power(features, p).mean(axis=0), 1/p)


def encode(crops, model):
    features = model.predict(crops)
    pooled_features = norm_pool(features)
    return pooled_features


def process_image(image_file):
    img = cv2.imread(image_file)
    if SCALE != 1:
        img = cv2.resize(img, None, fx=SCALE, fy=SCALE, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_norm = normalize_staining(img)

    for _ in range(AUGMENTATIONS_PER_IMAGE):
        img_aug = hematoxylin_eosin_aug(img_norm, low=COLOR_LO, high=COLOR_HI)
        single_image_crops = get_crops(img_aug, PATCH_SZ, PATCHES_PER_IMAGE)
        yield single_image_crops


def crops_gen(file_list):
    for i, (image_file, output_file) in enumerate(file_list):
        print("Crops generator:", i + 1)
        for crops in process_image(image_file):
            yield crops, output_file


def features_gen(crops_and_output_file, model):
    print("features gen");
    ts = time()
    current_file = None
    pooled_features = []
    i = 0
    for j, (crops, output_file) in enumerate(crops_and_output_file):
        print("cropping",j)
        if current_file is None:
            current_file = output_file
        features = encode(crops, model)
        if output_file == current_file:
            pooled_features.append(features)
        else:
            np.save(current_file, np.stack(pooled_features))
            pooled_features = [features]
            current_file = output_file
            average_time = int((time() - ts) / (i + 1))
            print("Feature generator: {}, {} sec/image.".format(i + 1, average_time))
            i += 1
    if len(pooled_features) > 0:
        np.save(current_file, np.stack(pooled_features))





In [3]:
DATASET_PATH = 'E:\Projects\HolisticClassification\TempDataset' # the dataset file or root folder path.
TEST_DATASET_PATH = 'E:\Projects\HolisticClassification\TempDataset\Test'

BATCH_SIZE = 4
MODE = 'folder' # or 'file', if you choose a plain text file (see above).
from PIL import Image
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time 

import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
#define 
def read_images_convert_to_jpeg(dataset_path = DATASET_PATH, mode = MODE, batch_size = BATCH_SIZE):
    imagepaths, labels = list(), list()
    if mode == 'folder':
        label = 0
        classes = [f.path for f in os.scandir(dataset_path) if f.is_dir() ] 
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            walk = os.walk(c_dir).__next__()
            for sample in walk[2]:
                if sample.endswith('.BMP') or sample.endswith('.bmp'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")
    for image_loc in imagepaths:
        new_img = Image.open(image_loc)
        save_loc = os.path.splitext(image_loc)[0] + ".jpeg"
        print(save_loc)
        new_img.save( save_loc, 'jpeg')
        

In [4]:
#read_images_convert_to_jpeg()

In [5]:
import numpy as np

import tensorflow as tf
class ResNet:
    __name__ = "ResNet"

    def __init__(self, batch_size=16):
        self.model = tf.contrib.keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', pooling="avg")
        self.batch_size = batch_size
        self.data_format =  tf.keras.backend.image_data_format()

    def predict(self, x):
        if self.data_format == "channels_first":
            x = x.transpose(0, 3, 1, 2)
        x = tf.contrib.keras.applications.resnet50.preprocess_input(x.astype(tf.keras.backend.floatx()))
        return self.model.predict(x, batch_size=16)

In [6]:
def threaded_generator(generator, num_cached=50):
    """Implements threaded generator to produce batches in background thread.
    # Arguments
        generator: an object exposing generator interface.
    # Yields
        Objects generated by generator.
    """
    from queue import Queue

    qu = Queue(maxsize=num_cached)
    sentinel = object()  # guaranteed unique reference

    # define producer (putting items into queue)
    def producer():
        for item in generator:
            qu.put(item)
        qu.put(sentinel)

    # start producer (in a background thread)
    import threading
    thread = threading.Thread(target=producer)
    thread.daemon = True
    thread.start()
    item = qu.get()
    while item is not sentinel:
        yield item
        qu.task_done()
        item = qu.get()


In [10]:
import os
from os.path import basename, join, exists
DATASET_PATH = 'E:\Projects\HolisticClassification\TempDataset' # the dataset file or root folder path.


NUM_CACHED = 160
NN_MODELS = [ResNet]
for SCALE in SCALES:
        print("SCALE:", SCALE)
        for NN_MODEL in NN_MODELS:
            print("NN_MODEL:", NN_MODEL.__name__)
            for PATCH_SZ in PATCH_SIZES:
                print("PATCH_SZ:", PATCH_SZ)
                model = NN_MODEL(batch_size=BATCH_SIZE)
                imagepaths, labels = list(), list()
                label = 0
                classes = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir() ] 
                for c in classes:
                    c_dir = os.path.join(DATASET_PATH, c)
                    walk = os.walk(c_dir).__next__()
                    for sample in walk[2]:
                        if sample.endswith('.jpeg') or sample.endswith('.jpg'):
                            imagepaths.append(os.path.join(c_dir, sample))
                            labels.append(label)
                            label += 1
                output_files = [join(DATASET_PATH, basename(f).replace("jpeg", "npy")) for f in imagepaths]
                file_list = zip(imagepaths, output_files)

                crops_and_output_file = crops_gen(file_list)
                #crops_and_output_file_ = threaded_generator(crops_and_output_file, num_cached=NUM_CACHED)
                features_gen(crops_and_output_file, model)

SCALE: 0.5
NN_MODEL: ResNet
PATCH_SZ: 224
features gen
Crops generator: 1


E:\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


cropping 0
cropping 1
cropping 2
cropping 3
cropping 4
cropping 5
cropping 6
cropping 7
cropping 8
cropping 9
cropping 10
cropping 11
cropping 12
cropping 13
cropping 14
cropping 15
cropping 16
cropping 17
cropping 18
cropping 19
cropping 20
cropping 21
cropping 22
cropping 23
cropping 24
cropping 25
cropping 26
cropping 27
cropping 28
cropping 29
cropping 30
cropping 31
cropping 32
cropping 33
cropping 34
cropping 35
cropping 36
cropping 37
cropping 38
cropping 39
cropping 40
cropping 41
cropping 42
cropping 43
cropping 44
cropping 45
cropping 46
cropping 47
cropping 48
cropping 49
Crops generator: 2
cropping 50
Feature generator: 1, 395 sec/image.
cropping 51
cropping 52
cropping 53
cropping 54
cropping 55
cropping 56
cropping 57
cropping 58
cropping 59
cropping 60
cropping 61
cropping 62
cropping 63
cropping 64
cropping 65
cropping 66
cropping 67
cropping 68
cropping 69
cropping 70
cropping 71
cropping 72
cropping 73
cropping 74
cropping 75
cropping 76
cropping 77
cropping 78
croppi

cropping 591
cropping 592
cropping 593
cropping 594
cropping 595
cropping 596
cropping 597
cropping 598
cropping 599
Crops generator: 13
cropping 600
Feature generator: 12, 398 sec/image.
cropping 601
cropping 602
cropping 603
cropping 604
cropping 605
cropping 606
cropping 607
cropping 608
cropping 609
cropping 610
cropping 611
cropping 612
cropping 613
cropping 614
cropping 615
cropping 616
cropping 617
cropping 618
cropping 619
cropping 620
cropping 621
cropping 622
cropping 623
cropping 624
cropping 625
cropping 626
cropping 627
cropping 628
cropping 629
cropping 630
cropping 631
cropping 632
cropping 633
cropping 634
cropping 635
cropping 636
cropping 637
cropping 638
cropping 639
cropping 640
cropping 641
cropping 642
cropping 643
cropping 644
cropping 645
cropping 646
cropping 647
cropping 648
cropping 649
Crops generator: 14
cropping 650
Feature generator: 13, 397 sec/image.
cropping 651
cropping 652
cropping 653
cropping 654
cropping 655
cropping 656
cropping 657
cropping 658


cropping 1156
cropping 1157
cropping 1158
cropping 1159
cropping 1160
cropping 1161
cropping 1162
cropping 1163
cropping 1164
cropping 1165
cropping 1166
cropping 1167
cropping 1168
cropping 1169
cropping 1170
cropping 1171
cropping 1172
cropping 1173
cropping 1174
cropping 1175
cropping 1176
cropping 1177
cropping 1178
cropping 1179
cropping 1180
cropping 1181
cropping 1182
cropping 1183
cropping 1184
cropping 1185
cropping 1186
cropping 1187
cropping 1188
cropping 1189
cropping 1190
cropping 1191
cropping 1192
cropping 1193
cropping 1194
cropping 1195
cropping 1196
cropping 1197
cropping 1198
cropping 1199
Crops generator: 25
cropping 1200
Feature generator: 24, 396 sec/image.
cropping 1201
cropping 1202
cropping 1203
cropping 1204
cropping 1205
cropping 1206
cropping 1207
cropping 1208
cropping 1209
cropping 1210
cropping 1211
cropping 1212
cropping 1213
cropping 1214
cropping 1215
cropping 1216
cropping 1217
cropping 1218
cropping 1219
cropping 1220
cropping 1221
cropping 1222
crop

...

cropping 2042
cropping 2043
cropping 2044
cropping 2045
cropping 2046
cropping 2047
cropping 2048
cropping 2049
Crops generator: 42
cropping 2050
Feature generator: 41, 408 sec/image.
cropping 2051
cropping 2052
cropping 2053
cropping 2054
cropping 2055
cropping 2056
cropping 2057
cropping 2058
cropping 2059
cropping 2060
cropping 2061
cropping 2062
cropping 2063
cropping 2064
cropping 2065
cropping 2066
cropping 2067
cropping 2068
cropping 2069
cropping 2070
cropping 2071
cropping 2072
cropping 2073
cropping 2074
cropping 2075
cropping 2076
cropping 2077
cropping 2078
cropping 2079
cropping 2080
cropping 2081
cropping 2082
cropping 2083
cropping 2084
cropping 2085
cropping 2086
cropping 2087
cropping 2088
cropping 2089
cropping 2090
cropping 2091
cropping 2092
cropping 2093
cropping 2094
cropping 2095
cropping 2096
cropping 2097
cropping 2098
cropping 2099
Crops generator: 43
cropping 2100
Feature generator: 42, 408 sec/image.
cropping 2101
cropping 2102
cropping 2103
cropping 2104
cr

cropping 2582
cropping 2583
cropping 2584
cropping 2585
cropping 2586
cropping 2587
cropping 2588
cropping 2589
cropping 2590
cropping 2591
cropping 2592
cropping 2593
cropping 2594
cropping 2595
cropping 2596
cropping 2597
cropping 2598
cropping 2599
Crops generator: 53
cropping 2600
Feature generator: 52, 404 sec/image.
cropping 2601
cropping 2602
cropping 2603
cropping 2604
cropping 2605
cropping 2606
cropping 2607
cropping 2608
cropping 2609
cropping 2610
cropping 2611
cropping 2612
cropping 2613
cropping 2614
cropping 2615
cropping 2616
cropping 2617
cropping 2618
cropping 2619
cropping 2620
cropping 2621
cropping 2622
cropping 2623
cropping 2624
cropping 2625
cropping 2626
cropping 2627
cropping 2628
cropping 2629
cropping 2630
cropping 2631
cropping 2632
cropping 2633
cropping 2634
cropping 2635
cropping 2636
cropping 2637
cropping 2638
cropping 2639
cropping 2640
cropping 2641
cropping 2642
cropping 2643
cropping 2644
cropping 2645
cropping 2646
cropping 2647
cropping 2648
crop

cropping 3122
cropping 3123
cropping 3124
cropping 3125
cropping 3126
cropping 3127
cropping 3128
cropping 3129
cropping 3130
cropping 3131
cropping 3132
cropping 3133
cropping 3134
cropping 3135
cropping 3136
cropping 3137
cropping 3138
cropping 3139
cropping 3140
cropping 3141
cropping 3142
cropping 3143
cropping 3144
cropping 3145
cropping 3146
cropping 3147
cropping 3148
cropping 3149
Crops generator: 64


C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
E:\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Degrees of freedom <= 0 for slice
  from ipykernel import kernelapp as app
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


LinAlgError: Array must not contain infs or NaNs